# Run PEST with BeoPEST

### This should be super easy...

In [1]:
%matplotlib inline
import os, shutil
import sys
sys.path.append("..")
import numpy as np
from IPython.display import Image
import pandas as pd
import matplotlib.pyplot as plt

import flopy as flopy
import pyemu
sys.path.insert(0, 'D:/spark-brc_gits/apexmf_git/apexmf_pkgs')
# from apexmf_pst_pkgs import apexmf_pst_utils, apexmf_pst_par
import apexmf_pst_utils

flopy is installed in C:\Users\seonggyu.park\Miniconda3\envs\sm_pest\lib\site-packages\flopy


# 1. Specify a path to your master

In [4]:
wd = "D:/Projects/Watersheds/Animas/Analysis/APEX-MODFLOWs/calibrations/v03"

In [5]:
os.chdir(wd)

# 2. Initial run for adjusting weights

In [84]:
pstfile = 'animas_pest.pst'
pstnosed = 'ani_pest_nosed.pst'

0

# Let's make sediment weight zeros

In [87]:
pst = pyemu.Pst(os.path.join(wd, pstfile))
obd = pst.observation_data
for i in range(len(obd)):
    if obd.iloc[i, 0][:3] == 'sed':
        obd.iloc[i, 2] = obd.iloc[i, 2] * 0
pst.observation_data = obd
pst.write(pstnosed)


noptmax:0, npar_adj:100, nnz_obs:917


In [88]:
os.system("start cmd /k i64pest {0}".format(pstnosed))

0

# 3. Reweight

In [103]:
casename = 'ani_pest_nosed.pst'
pestoutfile = 'ani_pest_nosed_rw.pst'
contrib = 2.01760E+05

In [94]:
os.system("start cmd /k i64pwtadj1 {0} {1} {2}".format(casename, pestoutfile, contrib))

0

In [95]:
# Stream 1 GW 0.1
# load the pre-constructed pst
pst = pyemu.Pst(os.path.join(wd, pestoutfile))
obd = pst.observation_data

In [96]:
for i in range(len(obd)):
    if obd.iloc[i, 0][:2] == 'gw':
        obd.iloc[i, 2] = obd.iloc[i, 2] * 0.5

pst.observation_data = obd
pst.write(pestoutfile)

noptmax:50, npar_adj:100, nnz_obs:917


In [104]:
wd

'D:/Projects/Watersheds/Animas/Analysis/APEX-MODFLOWs/calibrations/v03'

In [102]:
pestoutfile

'ani_pest_nosed_rw02.pst'

In [105]:
pst = pyemu.Pst(os.path.join(wd, pestoutfile))
pestoutfile02 = 'ani_pest_nosed_rw02.pst'
pst.write(pestoutfile02)

noptmax:50, npar_adj:100, nnz_obs:917


Go check the pst file

# 4. Run Parallel PEST with BeoPEST

In [ ]:
execute_beopest?

In [106]:
apexmf_pst_utils.execute_beopest(
    wd, pestoutfile02,
    reuse_workers=True,
    num_workers=12,
    # restart=True,
    copy_files=['MODFLOW/mf_1000.nwt', 'APEXCONT.DAT']
    )

# 5. Run or add workers separately

In [ ]:
execute_workers?

In [11]:
apexmf_pst_utils.execute_workers(
    wd, pestoutfile, "172.20.1.9", num_workers=5, start_id=28,
    # worker_root='..', port=4005,
    reuse_workers=True,
    # copy_files=['model.in', 'model.in.tpl']
    )

- An easy way to formulate a list of prior parameter standard deviations (this being one option for filling a parameter uncertainty file) is to divide the difference between these bounds (or their logs for parameters that are log-transformed) by 4.0. 
- This procedure relies on the the implicit assumption that parameters are (log)normally distributed and that their bounds span their 95% confidence intervals.

In [5]:
create_param_unc?

Signature: create_param_unc(pst_file, unc_file=None, sampl_n=None)
Docstring:
create a parameter uncertainty file from an existing *.pst file

Args:
    - pst_file (`str`): path and name of existing *.pst file
    - unc_file (`str`): name of parameter uncertainty file
                        If `None`, then `param.unc` is used.
                        Defult is `None`.
    - sampl_n ('int'): sample number from normal distribution
                        If `None`, then `1000` is used.
                        Defult is `None`.
Returns:
    `pandas.DataFrame`: a dataframe of log standard deviation for each parameter
    `param.unc file`

Example:
    sm_pst_stats.create_param_unc('my.pst', 'my.unc', 2000)
File:      e:\qsm_300_zon_p\sm_pst_pkgs\sm_pst_stats.py
Type:      function


In [12]:
create_param_unc("qsm_reweight.pst")

param.unc file has been created...


,parlbnd,parubnd,mu,sigma,lower_95,upper_95,log_sd
parnme,,,,,,,
ed,1.0000000000e+00,1.0000000000e+02,50.500000,24.750000,1.932049,99.067951,0.427479
kc,2.5924320000e-03,2.5924320000e+01,12.963456,6.480432,0.246636,25.680276,0.504386
kdfdc,2.0347200000e-03,2.0347200000e+01,10.174617,5.086291,0.193577,20.155658,0.504386
kked,5.1840000000e-03,5.1840000000e+01,25.922592,12.958704,0.493190,51.351994,0.504386
kms,1.2960000000e-03,1.2960000000e+01,6.480648,3.239676,0.123297,12.837999,0.504386
kpw,2.5924320000e-03,2.5924320000e+01,12.963456,6.480432,0.246636,25.680276,0.504386
qal,2.9600000000e+01,1.0000000000e+03,514.800000,242.600000,38.735962,990.864038,0.351975
qt,5.9600000000e+01,1.5000000000e+03,779.800000,360.100000,73.160840,1486.439160,0.326967
riv_bot,3.5000000000e-01,6.5000000000e-01,0.500000,0.075000,0.352824,0.647176,0.065866


In [16]:
test = create_param_unc("qsm_reweight.pst")

param.unc file has been created...


#### Calibration with SVD

In [5]:
execute_beopest(wd, "qsm_svd.pst", 10)

#### Calibration with SVD-Reg

In [4]:
execute_beopest(wd, "svd-reg.pst", 15)

#### Calibration

In [5]:
execute_beopest(wd, "qsm_reweight.pst", 10)

In [6]:
import socket
socket.gethostname()

'ARSTXTEM4119734'